# 02 – Network Properties of the Collaboration Graph

This notebook extends the exploratory analysis carried out in `01_exploration.ipynb` by computing and inspecting more advanced network properties of the collaboration graph.

It focuses on the **largest connected component (LCC)** and analyzes:

- Path length characteristics (average shortest path length, diameter approximation)
- Assortativity by degree
- Centrality measures (degree, betweenness, eigenvector)
- Top nodes according to each centrality

All results are saved in the `../results/` directory so they can be referenced in the research paper (Methods / Network Analysis section).

## 1. Setup and Imports

We assume the same directory structure as before:

```text
upe-ppgec-netsci-2025-1-projeto-icbvo/
├── data/
│   └── collaboration.edgelist.txt
├── notebooks/
├── results/
└── gnn/
```

We will load the edge list and build the graph again using NetworkX. We will also load the `graph_summary.json` generated in `01_exploration.ipynb` if available.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx
from pathlib import Path
import json

%matplotlib inline

DATA_PATH = Path("../data/collaboration.edgelist.txt")
RESULTS_DIR = Path("../results")
RESULTS_DIR.mkdir(parents=True, exist_ok=True)

SUMMARY_PATH = RESULTS_DIR / "graph_summary.json"

DATA_PATH, RESULTS_DIR, SUMMARY_PATH

## 2. Load Edge List and Graph

We load the edge list and construct an undirected NetworkX graph as before. This ensures that all metrics here are consistent with the previous notebook.

In [ ]:
if not DATA_PATH.exists():
    raise FileNotFoundError(f"Edge list file not found: {DATA_PATH}")

df_edges = pd.read_csv(DATA_PATH, sep=r"\s+", header=None, names=["u", "v"])
df_edges.head()

In [ ]:
G = nx.from_pandas_edgelist(df_edges, source="u", target="v")
print(G)
print(f"Number of nodes: {G.number_of_nodes()}")
print(f"Number of edges: {G.number_of_edges()}")

If available, we also load the basic summary computed in `01_exploration.ipynb` for reference.

In [ ]:
if SUMMARY_PATH.exists():
    with open(SUMMARY_PATH, "r") as f:
        summary_prev = json.load(f)
    summary_prev
else:
    summary_prev = None
    print("No previous summary found at", SUMMARY_PATH)

## 3. Largest Connected Component (LCC)

Many global metrics are best interpreted (and sometimes only defined) on connected graphs. We therefore focus on the **largest connected component (LCC)** of the collaboration network.

This is also coherent with the idea of analyzing the giant component in complex networks.

In [ ]:
components = list(nx.connected_components(G))
n_components = len(components)
lcc_nodes = max(components, key=len)
LCC = G.subgraph(lcc_nodes).copy()

print(f"Number of connected components: {n_components}")
print(f"LCC number of nodes: {LCC.number_of_nodes()}")
print(f"LCC number of edges: {LCC.number_of_edges()}")
print(f"Fraction of nodes in LCC: {LCC.number_of_nodes() / G.number_of_nodes():.4f}")

## 4. Path Length Characteristics in the LCC

We approximate the distribution of shortest path lengths in the LCC by:

- Computing exact average shortest path length on the LCC (if feasible), or
- Sampling a subset of nodes if the LCC is very large.

We also estimate the effective diameter (e.g., 90th percentile of shortest path lengths in the sample).

In [ ]:
n_lcc_nodes = LCC.number_of_nodes()
print(f"LCC size: {n_lcc_nodes} nodes")

# If the LCC is not too large, we can compute the average shortest path length directly.
if n_lcc_nodes <= 5000:
    avg_spl = nx.average_shortest_path_length(LCC)
    print(f"Average shortest path length (exact): {avg_spl:.4f}")
    sampled_path_lengths = None
else:
    print("LCC is large; approximating shortest path lengths by sampling nodes...")
    import random
    random.seed(42)
    sample_size = 500  # number of source nodes to sample
    nodes_list = list(LCC.nodes())
    sampled_nodes = random.sample(nodes_list, min(sample_size, n_lcc_nodes))

    sampled_path_lengths = []
    for s in sampled_nodes:
        lengths = nx.single_source_shortest_path_length(LCC, s)
        sampled_path_lengths.extend(list(lengths.values()))

    sampled_path_lengths = np.array(sampled_path_lengths)
    avg_spl = sampled_path_lengths.mean()
    print(f"Average shortest path length (sampled): {avg_spl:.4f}")

avg_spl

In [ ]:
if LCC.number_of_nodes() <= 5000:
    # For a smaller LCC, we can also compute the diameter exactly
    try:
        diameter = nx.diameter(LCC)
        print(f"Graph diameter (exact): {diameter}")
    except Exception as e:
        print("Could not compute exact diameter:", e)
        diameter = None
else:
    diameter = None
    if isinstance(sampled_path_lengths, np.ndarray):
        p90 = np.percentile(sampled_path_lengths, 90)
        print(f"Approximate 90th percentile shortest path length (effective diameter): {p90:.4f}")
    else:
        print("No sampled path lengths available for diameter approximation.")

diameter

## 5. Assortativity by Degree

We compute the **degree assortativity coefficient** of the LCC:

- Positive assortativity indicates that high-degree nodes tend to connect to other high-degree nodes.
- Negative assortativity indicates that high-degree nodes tend to connect to low-degree nodes.
- Values close to zero suggest little or no degree correlation.

In [ ]:
degree_assortativity = nx.degree_assortativity_coefficient(LCC)
print(f"Degree assortativity coefficient (LCC): {degree_assortativity:.4f}")

## 6. Centrality Measures

We focus on three centrality measures computed on the LCC:

- **Degree centrality**: normalized degree of each node.
- **Betweenness centrality**: fraction of shortest paths that pass through the node.
- **Eigenvector centrality**: importance of a node based on the importance of its neighbors.

Note: Betweenness and eigenvector centralities can be computationally intensive on large graphs; in that case we may approximate or rely on tolerances/iterations.

In [ ]:
# Degree centrality (fast)
deg_centrality = nx.degree_centrality(LCC)

# Betweenness centrality (may be expensive; we can sample nodes if needed)
n_lcc_nodes = LCC.number_of_nodes()
if n_lcc_nodes <= 5000:
    betw_centrality = nx.betweenness_centrality(LCC, normalized=True)
else:
    # Approximate betweenness using k sampled nodes
    import random
    random.seed(42)
    k = 500
    sample_nodes = random.sample(list(LCC.nodes()), min(k, n_lcc_nodes))
    betw_centrality = nx.betweenness_centrality(LCC, k=sample_nodes, normalized=True)

# Eigenvector centrality (power iteration)
try:
    eig_centrality = nx.eigenvector_centrality(LCC, max_iter=1000, tol=1e-06)
except Exception as e:
    print("Eigenvector centrality did not converge:", e)
    eig_centrality = None

len(deg_centrality), len(betw_centrality), (len(eig_centrality) if eig_centrality is not None else None)

### 6.1. Top Nodes by Degree Centrality

We list the top 10 nodes according to degree centrality in the LCC.

In [ ]:
def top_k_centrality(cent_dict, k=10):
    return sorted(cent_dict.items(), key=lambda x: x[1], reverse=True)[:k]

top10_deg = top_k_centrality(deg_centrality, k=10)
top10_deg

### 6.2. Top Nodes by Betweenness Centrality

Betweenness centrality highlights nodes that act as bridges in the network.

In [ ]:
top10_betw = top_k_centrality(betw_centrality, k=10)
top10_betw

### 6.3. Top Nodes by Eigenvector Centrality

Eigenvector centrality captures nodes that are connected to other important nodes. If the computation did not converge, this may be `None`.

In [ ]:
if eig_centrality is not None:
    top10_eig = top_k_centrality(eig_centrality, k=10)
    top10_eig
else:
    print("Eigenvector centrality is not available.")
    top10_eig = None

## 7. Export Centrality Results

We aggregate centrality metrics into a single `pandas.DataFrame` and export them as a CSV file to the `../results/` directory.

This file can later be used to:

- Inspect specific nodes of interest
- Compare centrality profiles
- Support interpretations in the research paper (e.g., identifying hub authors or bridging authors)

In [ ]:
nodes_lcc = list(LCC.nodes())

df_centrality = pd.DataFrame({
    "node": nodes_lcc,
    "degree_centrality": [deg_centrality[n] for n in nodes_lcc],
    "betweenness_centrality": [betw_centrality[n] for n in nodes_lcc],
})

if eig_centrality is not None:
    df_centrality["eigenvector_centrality"] = [eig_centrality[n] for n in nodes_lcc]
else:
    df_centrality["eigenvector_centrality"] = np.nan

centrality_csv_path = RESULTS_DIR / "centrality_lcc.csv"
df_centrality.to_csv(centrality_csv_path, index=False)
centrality_csv_path, df_centrality.head()

## 8. Final Summary

We aggregate key properties of the LCC and save them in a JSON file named `network_properties_lcc.json` inside the `../results/` directory.

These properties include:

- LCC size (nodes, edges)
- Fraction of nodes in the LCC
- Average shortest path length (exact or sampled)
- Diameter (exact or approximated) when available
- Degree assortativity coefficient

This file can be directly referenced in the *Network Description* section of the paper.

In [ ]:
props_lcc = {
    "lcc_n_nodes": int(LCC.number_of_nodes()),
    "lcc_n_edges": int(LCC.number_of_edges()),
    "lcc_fraction_of_total_nodes": float(LCC.number_of_nodes() / G.number_of_nodes()),
    "average_shortest_path_length": float(avg_spl),
    "diameter_exact_or_none": int(diameter) if diameter is not None else None,
    "degree_assortativity": float(degree_assortativity),
}

props_path = RESULTS_DIR / "network_properties_lcc.json"
with open(props_path, "w") as f:
    json.dump(props_lcc, f, indent=4)

print(f"LCC properties saved to: {props_path}")
props_lcc